# DATASUS Data Collector using PySUS

##### use this on google colab or use on a linux distro, windows stopped working

In [68]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [69]:
import pkg_resources

# Package checking list
packages_to_check = [
    {
     'name': 'pandas',
     'version':'1.5.3'
     },
    {
     'name': 'pySUS',
     'version': '0.9.4'
     },
    ]

# check if necessary packages are installed, and install if not
for package in packages_to_check:
    print(package)
    try:
        pkg_resources.get_distribution(package['name'])
        print(f"{package['name']} is installed.")
    except pkg_resources.DistributionNotFound:
        print(f"{package['name']} is not installed. Installing:")
        %pip install {package['name']}=={package['version']}

{'name': 'pandas', 'version': '1.5.3'}
pandas is installed.
{'name': 'pySUS', 'version': '0.9.4'}
pySUS is installed.


In [70]:
import pandas as pd
from pysus.online_data.SIM import download

In [71]:
import os

drive_dir_path = '/content/drive/My Drive/'
dir_path_to_save = 'data_storage/0_datasus_csvs/'

user_dir_path = ''

if user_dir_path:
    full_save_path = os.path.join(drive_dir_path, user_dir_path, dir_path_to_save)
else:
    full_save_path = os.path.join(drive_dir_path, dir_path_to_save)

print(f'Location that the csvs will be saved:\n {full_save_path}')

if not os.path.exists(full_save_path):
    os.makedirs(full_save_path)

Location that the csvs will be saved:
 /content/drive/My Drive/.Mestrado/Orientação/.Datasus Data Reader/data_storage/0_datasus_csvs/


In [72]:
import datetime

# Definition of all states from all regions in Brazil
states_sul = ["PR", "RS", "SC"]
states_sudeste = ["ES", "MG", "RJ", "SP"]
states_nordeste = ["AL", "BA", "CE", "MA", "PE", "PI", "PB", "RN", "SE"]
states_norte = ["AC", "AM", "AP", "PA", "RO", "RR", "TO"]
states_centrooeste = ["DF", "GO", "MS", "MT"]

# Initialization of all_states, that contain all states in Brazil
all_states = states_sul + states_sudeste + states_nordeste + states_norte + states_centrooeste

# Initialization of states_to_download, which will determine which states'
# data will be downloaded
states_to_download = all_states
print(f"States that will be downloaded:\n{states_to_download}\n")

# Edit here to change the first year
first_year = 1996
# Current year is an open interval
current_year = datetime.datetime.now().year
print(f"Current year: {current_year}")

# Years that will be downloaded, [first_year, current_year) interval
years_to_download = [x for x in range(first_year, current_year)]
print(f"Years that will be downloaded:\n{years_to_download}")

States that will be downloaded:
['PR', 'RS', 'SC', 'ES', 'MG', 'RJ', 'SP', 'AL', 'BA', 'CE', 'MA', 'PE', 'PI', 'PB', 'RN', 'SE', 'AC', 'AM', 'AP', 'PA', 'RO', 'RR', 'TO', 'DF', 'GO', 'MS', 'MT']

Current year: 2024
Years that will be downloaded:
[1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]


In [73]:
# Iteration of years
for i_year in years_to_download:
  # Initialization of data dictionary
  states_dict = {}

  # Iteration of states
  for i_state in states_to_download:
    # Store (state, year) data in states_dict
    states_dict[i_state, i_year] = download(i_state, i_year)
    print(str(i_state) + " database from " + str(i_year) + " downloaded!")

  print('states_dict result:', '\n', states_dict, '\n')

  if states_dict[list(states_dict.keys())[0]] == ():
    print("states_dict result doesn't have valid years, skipping this year.")
    continue

  # Initialization of data_df, which contains the states data in the current iterated year
  data_df = pd.concat({k: pd.DataFrame.from_dict(pd.read_parquet(v)) for k, v in states_dict.items()}, axis=0).reset_index()
  # For PySUS version 0.10^
  # data_df = pd.concat({k: pd.DataFrame.from_dict(v.to_dataframe()) for k, v in states_dict.items()}, axis=0).reset_index()

  # Trailing whitespace striping otherwise comparisons don't work 💀
  data_df["CAUSABAS"] = data_df["CAUSABAS"].str.strip()

  # Keep only codes that correspond to suicide
  icd10_filter_list = ["X{}".format(x) for x in range(600, 850)]
  suicides_df = data_df[data_df["CAUSABAS"].isin(icd10_filter_list)]

  # Renaming columns to names that make sense
  suicides_df.rename(columns={"level_0": "ESTADO", "level_1": "ANO"}, inplace=True)

  # Upload to chosen folder
  suicides_df.to_csv(full_save_path + str(i_year) + '.csv', index=False)

  # Use parquets if RAM explodes
  # suicides_df.to_parquet(full_save_path + str(i_year) + '.parquet', index=False)

# End of year iteration

print("Download and upload finished.")

PR database from 1996 downloaded!
RS database from 1996 downloaded!
SC database from 1996 downloaded!
ES database from 1996 downloaded!
MG database from 1996 downloaded!
RJ database from 1996 downloaded!
SP database from 1996 downloaded!
AL database from 1996 downloaded!
BA database from 1996 downloaded!
CE database from 1996 downloaded!
MA database from 1996 downloaded!
PE database from 1996 downloaded!
PI database from 1996 downloaded!
PB database from 1996 downloaded!
RN database from 1996 downloaded!
SE database from 1996 downloaded!
AC database from 1996 downloaded!
AM database from 1996 downloaded!
AP database from 1996 downloaded!
PA database from 1996 downloaded!
RO database from 1996 downloaded!
RR database from 1996 downloaded!
TO database from 1996 downloaded!
DF database from 1996 downloaded!
GO database from 1996 downloaded!
MS database from 1996 downloaded!
MT database from 1996 downloaded!
states_dict result: 
 {('PR', 1996): '/root/pysus/DOPR1996.parquet', ('RS', 1996):

/tmp/ipykernel_1092/51939402.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  suicides_df.rename(columns={"level_0": "ESTADO", "level_1": "ANO"}, inplace=True)


PR database from 1997 downloaded!
RS database from 1997 downloaded!
SC database from 1997 downloaded!
ES database from 1997 downloaded!
MG database from 1997 downloaded!
RJ database from 1997 downloaded!
SP database from 1997 downloaded!
AL database from 1997 downloaded!
BA database from 1997 downloaded!
CE database from 1997 downloaded!
MA database from 1997 downloaded!
PE database from 1997 downloaded!
PI database from 1997 downloaded!
PB database from 1997 downloaded!
RN database from 1997 downloaded!
SE database from 1997 downloaded!
AC database from 1997 downloaded!
AM database from 1997 downloaded!
AP database from 1997 downloaded!
PA database from 1997 downloaded!
RO database from 1997 downloaded!
RR database from 1997 downloaded!
TO database from 1997 downloaded!
DF database from 1997 downloaded!
GO database from 1997 downloaded!
MS database from 1997 downloaded!
MT database from 1997 downloaded!
states_dict result: 
 {('PR', 1997): '/root/pysus/DOPR1997.parquet', ('RS', 1997):

/tmp/ipykernel_1092/51939402.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  suicides_df.rename(columns={"level_0": "ESTADO", "level_1": "ANO"}, inplace=True)


PR database from 1998 downloaded!
RS database from 1998 downloaded!
SC database from 1998 downloaded!
ES database from 1998 downloaded!
MG database from 1998 downloaded!
RJ database from 1998 downloaded!
SP database from 1998 downloaded!
AL database from 1998 downloaded!
BA database from 1998 downloaded!
CE database from 1998 downloaded!
MA database from 1998 downloaded!
PE database from 1998 downloaded!
PI database from 1998 downloaded!
PB database from 1998 downloaded!
RN database from 1998 downloaded!
SE database from 1998 downloaded!
AC database from 1998 downloaded!
AM database from 1998 downloaded!
AP database from 1998 downloaded!
PA database from 1998 downloaded!
RO database from 1998 downloaded!
RR database from 1998 downloaded!
TO database from 1998 downloaded!
DF database from 1998 downloaded!
GO database from 1998 downloaded!
MS database from 1998 downloaded!
MT database from 1998 downloaded!
states_dict result: 
 {('PR', 1998): '/root/pysus/DOPR1998.parquet', ('RS', 1998):

...

PR database from 2021 downloaded!
RS database from 2021 downloaded!
SC database from 2021 downloaded!
ES database from 2021 downloaded!
MG database from 2021 downloaded!
RJ database from 2021 downloaded!
SP database from 2021 downloaded!
AL database from 2021 downloaded!
BA database from 2021 downloaded!
CE database from 2021 downloaded!
MA database from 2021 downloaded!
PE database from 2021 downloaded!
PI database from 2021 downloaded!
PB database from 2021 downloaded!
RN database from 2021 downloaded!
SE database from 2021 downloaded!
AC database from 2021 downloaded!
AM database from 2021 downloaded!
AP database from 2021 downloaded!
PA database from 2021 downloaded!
RO database from 2021 downloaded!
RR database from 2021 downloaded!
TO database from 2021 downloaded!
DF database from 2021 downloaded!
GO database from 2021 downloaded!
MS database from 2021 downloaded!
MT database from 2021 downloaded!
states_dict result: 
 {('PR', 2021): '/root/pysus/DOPR2021.parquet', ('RS', 2021):

/tmp/ipykernel_1092/51939402.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  suicides_df.rename(columns={"level_0": "ESTADO", "level_1": "ANO"}, inplace=True)


PR database from 2022 downloaded!
RS database from 2022 downloaded!
SC database from 2022 downloaded!
ES database from 2022 downloaded!
MG database from 2022 downloaded!
RJ database from 2022 downloaded!
SP database from 2022 downloaded!
AL database from 2022 downloaded!
BA database from 2022 downloaded!
CE database from 2022 downloaded!
MA database from 2022 downloaded!
PE database from 2022 downloaded!
PI database from 2022 downloaded!
PB database from 2022 downloaded!
RN database from 2022 downloaded!
SE database from 2022 downloaded!
AC database from 2022 downloaded!
AM database from 2022 downloaded!
AP database from 2022 downloaded!
PA database from 2022 downloaded!
RO database from 2022 downloaded!
RR database from 2022 downloaded!
TO database from 2022 downloaded!
DF database from 2022 downloaded!
GO database from 2022 downloaded!
MS database from 2022 downloaded!
MT database from 2022 downloaded!
states_dict result: 
 {('PR', 2022): '/root/pysus/DOPR2022.parquet', ('RS', 2022):

/tmp/ipykernel_1092/51939402.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  suicides_df.rename(columns={"level_0": "ESTADO", "level_1": "ANO"}, inplace=True)


PR database from 2023 downloaded!
RS database from 2023 downloaded!
SC database from 2023 downloaded!
ES database from 2023 downloaded!
MG database from 2023 downloaded!
RJ database from 2023 downloaded!
SP database from 2023 downloaded!
AL database from 2023 downloaded!
BA database from 2023 downloaded!
CE database from 2023 downloaded!
MA database from 2023 downloaded!
PE database from 2023 downloaded!
PI database from 2023 downloaded!
PB database from 2023 downloaded!
RN database from 2023 downloaded!
SE database from 2023 downloaded!
AC database from 2023 downloaded!
AM database from 2023 downloaded!
AP database from 2023 downloaded!
PA database from 2023 downloaded!
RO database from 2023 downloaded!
RR database from 2023 downloaded!
TO database from 2023 downloaded!
DF database from 2023 downloaded!
GO database from 2023 downloaded!
MS database from 2023 downloaded!
MT database from 2023 downloaded!
states_dict result: 
 {('PR', 2023): (), ('RS', 2023): (), ('SC', 2023): (), ('ES'